<a href="https://colab.research.google.com/github/nmcardoso/splus-model/blob/master/splus_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%tensorflow_version 1.x 
import random
import tensorflow as tf
import numpy as np
from keras import backend as K
import os
import glob
import multiprocessing as mp
import shutil
import pandas as pd

from keras import layers
from keras import models
from keras import optimizers
from keras import utils
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import VGG16

import matplotlib.pyplot as plt

from progressbar import progressbar

!pip3 install git+https://github.com/nmcardoso/fitsbook-python
import fitsbook as fb

  Cloning https://github.com/nmcardoso/fitsbook-python to /tmp/pip-req-build-0cegj1xm
  Running command git clone -q https://github.com/nmcardoso/fitsbook-python /tmp/pip-req-build-0cegj1xm
  Created wheel for fitsbook: filename=fitsbook-0.1-cp36-none-any.whl size=2181 sha256=7403215adb4575963a67f21a26015b88420ec25b9300ad028efe6df834f0e557
  Stored in directory: /tmp/pip-ephem-wheel-cache-ewji4laa/wheels/99/2a/a0/b37947be424acac83a373b37b38a2bfaa4557b049c19426655
Successfully built fitsbook


In [2]:
# Reprodutibilidade
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

os.environ['PYTHONHASHSEED'] = '12'
np.random.seed(123)
random.seed(1234)
tf.set_random_seed(12345)

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, 
#                               inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

print('Sementes Plantadas!')

Sementes Plantadas!


In [0]:
!unzip /content/drive/'My Drive'/datasets/stamps_scaled.zip -d dataset

In [0]:
!tar -zxvf /content/drive/'My Drive'/datasets/dataset.tar.gz

In [0]:
!cp /content/drive/'My Drive'/datasets/stamps_scaled.csv /content/

In [0]:
class NpySequence(utils.Sequence):
  """Generate data for Keras"""
  def __init__(self, files, labels, classes, image_size=(256, 256), channels=3, batch_size=32):
    self.files = files
    self.labels = self.__to_categorical(labels, classes)
    self.image_size = image_size
    self.channels = channels
    self.batch_size = batch_size
    self.indexes = [i for i in range(len(files))]
    np.random.shuffle(self.indexes)
    # self.on_epoch_end()
  
  def __len__(self):
    """The number of batches per epoch"""
    return int(np.floor(len(self.files) / self.batch_size))
  
  def __getitem__(self, index):
    """Generate one batch of data"""
    partition = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    X, Y = self.__data_generation(partition)
    return X, Y
  
  def on_epoch_end(self):
    """Shuffle files list and remap labels"""
    np.random.shuffle(self.indexes)
    
  def __to_categorical(self, labels, classes):
    """Labels to numbers"""
    classes_dict = {c:i for i,c in enumerate(classes)}
    return [classes_dict[l] for l in labels]

  def __data_generation(self, partition):
    """Load npy file and associate with a label"""
    X = np.empty((self.batch_size, *self.image_size, self.channels))
    Y = np.empty((self.batch_size,), dtype=int)

    for i, index in enumerate(partition):
      X[i,] = np.load(self.files[index])
      Y[i] = self.labels[index]

    return X, Y

In [0]:
def preprocess():
  global train_generator, test_generator, val_generator
  df = pd.read_csv('/content/stamps_scaled.csv')
  E = df[df.CLASS == 'E']
  S = df[df.CLASS == 'S']

  # train_df = pd.concat([E[0:3127], S[0:951]])
  train_df = pd.concat([E[0:800], S[0:600]])
  train_X = [f'/content/dataset/{f}.npy' for f in train_df.ID.tolist()]
  train_Y = train_df.CLASS.tolist()

  # val_df = pd.concat([E[3128:5004], S[952:1522]])
  val_df = pd.concat([E[801:1300], S[601:1100]])
  val_X = [f'/content/dataset/{f}.npy' for f in val_df.ID.tolist()]
  val_Y = val_df.CLASS.tolist()

  # test_df = pd.concat([E[5005:], S[1523:]])
  test_df = pd.concat([E[1301:1700], S[1101:1500]])
  test_X = [f'/content/dataset/{f}.npy' for f in test_df.ID.tolist()]
  test_Y = test_df.CLASS.tolist()

  train_generator = NpySequence(train_X, train_Y, ['E', 'S'])
  val_generator = NpySequence(val_X, val_Y, ['E', 'S'])
  test_generator = NpySequence(test_X, test_Y, ['E', 'S'])

def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
  # 1x1 conv
  conv1 = layers.Conv2D(f1, (1, 1), padding='same', activation='relu')(layer_in)

  # 3x3 conv
  conv3 = layers.Conv2D(f2_in, (1, 1), padding='same', activation='relu')(layer_in)
  conv3 = layers.Conv2D(f2_out, (3, 3), padding='same', activation='relu')(layer_in)

  # 5x5 conv
  conv5 = layers.Conv2D(f3_in, (1, 1), padding='same', activation='relu')(layer_in)
  conv5 = layers.Conv2D(f3_out, (5, 5), padding='same', activation='relu')(conv5)

  # 3x3 max pooling
  pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(layer_in)
  pool = layers.Conv2D(f4_out, (1, 1), padding='same', activation='relu')(pool)

  # concatenate filters, assumes filters/channels last
  layer_out = layers.concatenate([conv1, conv3, conv5, pool], axis=-1)
  return layer_out

def compile_inception_model(image_size):
  input_layer = layers.Input(shape=image_size + (3,))
  conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
  pool1 = layers.MaxPool2D((2, 2))(conv1)
  conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
  pool2 = layers.MaxPool2D((2, 2))(conv2)
  iblock1 = inception_module(input_layer, 32, 48, 64, 8, 16, 16)
  iblock2 = inception_module(iblock1, 64, 48, 96, 16, 48, 32)
  flatten = layers.Flatten()(iblock2)
  dense1 = layers.Dense(1024, activation='relu')(flatten)
  dropout1 = layers.Dropout(.3)(dense1)
  dense2 = layers.Dense(512, activation='relu')(dropout1)
  dropout2 = layers.Dropout(.3)(dense2)
  dense3 = layers.Dense(256, activation='relu')(dropout2)
  dropout3 = layers.Dropout(.3)(dense3)
  output = layers.Dense(1, activation='sigmoid')(dropout3)
  model = models.Model(inputs=input_layer, outputs=output)
  model.name = 'splus_inception'
  utils.plot_model(model, to_file='model.png')
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

def compile_inception_pretrained(image_size):
  base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(1024, activation='relu')(x)
  output = layers.Dense(1, activation='sigmoid')(x)

  model = models.Model(inputs=base_model.input, outputs=output)
  model.name = 'splus_inception_pretrained'

  for layer in base_model.layers:
    layer.trainable = False
  
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])
  
  return model

def compile_model(image_size):
  model = models.Sequential()
  model.name = 'splus_convolutional'
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size + (3,))))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizers.AdaDelta(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

def fit(model, epochs=1):
  global history, train_generator, val_generator
  history = model.fit_generator(
      train_generator,
      # steps_per_epoch=80,
      # validation_steps=30,
      epochs=epochs,
      validation_data=val_generator,
      callbacks=[fb.callbacks.FitsbookCallback()],
      # use_multiprocessing=True,
      # workers=2
  )

In [20]:
preprocess()
m = compile_inception_pretrained((256, 256))
fit(m, 300)

Epoch 1/300
43/43 [==============================] - 97s 2s/step - loss: 0.7006 - acc: 0.5640 - val_loss: 0.6963 - val_acc: 0.5282
Epoch 2/300
43/43 [==============================] - 21s 481ms/step - loss: 0.6740 - acc: 0.5952 - val_loss: 0.7521 - val_acc: 0.5020
Epoch 3/300
43/43 [==============================] - 17s 406ms/step - loss: 0.6563 - acc: 0.6221 - val_loss: 0.7506 - val_acc: 0.5030
Epoch 4/300
43/43 [==============================] - 17s 405ms/step - loss: 0.6506 - acc: 0.6199 - val_loss: 0.7329 - val_acc: 0.5000
Epoch 5/300
43/43 [==============================] - 17s 403ms/step - loss: 0.6427 - acc: 0.6337 - val_loss: 0.7664 - val_acc: 0.5040
Epoch 6/300
42/43 [============================>.] - ETA: 0s - loss: 0.6440 - acc: 0.6339

KeyboardInterrupt: ignored